In [4]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [3]:
from langchain_ollama import ChatOllama
qwen_llm = ChatOllama(base_url="http://localhost:11434",
                       model="qwen2.5:latest",
                       temperature=0.5,
                       max_tokens=250)
llama_llm = ChatOllama(base_url="http://localhost:11434",
                       model="Llama3.2:latest",
                       temperature=0.5,
                       max_tokens=250)

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt1 = ChatPromptTemplate([
    ("system", "You are an expert on LLMs."),
    ("user", "What are the advantages of using LLMs in {env}?"),
])
prompt2 = ChatPromptTemplate.from_template("""
                                           Analyse the response and get me just headings from {response}?
                                           
                                           Response should be in bullet points.
                                           """)

In [5]:
# Using RunnableLambda to choose LLM based on response length
from langchain_core.runnables import RunnableLambda
def choose_llm(response_str) -> ChatOllama:
    if len(str(response_str)) > 300:
        return llama_llm
    return qwen_llm

runnable_llm = RunnableLambda(choose_llm)

In [6]:
from langchain_core.output_parsers import StrOutputParser
chain1 = prompt1 | runnable_llm | StrOutputParser()
chain2 = {"response": chain1} | prompt2 | runnable_llm | StrOutputParser()
response = chain2.invoke({"env": "Local System"})
print("Response:\n", response)


Response:
 Here are the headings from the response:

* Advantages of Using Large Language Models (LLMs) Locally
  • Enhanced Privacy
  • Reduced Latency
  • Offline Capabilities
  • Cost Efficiency
  • Customization and Control
  • Scalability
  • Compliance and Regulations
  • Performance Optimization
  • Security
  • Real-Time Analysis

* Considerations for Effective Deployment
  • Computational Requirements


In [ ]:
# Using @Chain decorator to define the chain
from langchain_core.runnables import chain
@chain
def choose_llm(response_str) -> ChatOllama:
    if len(str(response_str)) > 300:
        return llama_llm
    return qwen_llm

In [8]:
from langchain_core.output_parsers import StrOutputParser
chain1 = prompt1 | qwen_llm | StrOutputParser()
chain2 = {"response": chain1} | prompt2 | choose_llm | StrOutputParser()
response = chain2.invoke({"env": "Local System"})
print("Response:\n", response)

Response:
 Here are the headings from the response:

• **Privacy and Security**
• **Latency Reduction**
• **Offline Capabilities**
• **Customization and Control**
• **Cost Efficiency**
• **Scalability**
• **Compliance and Regulations**
• **Performance Optimization**
• **Resource Management**
• **Custom Training and Fine-Tuning**
